# Feed Forward Neural Network MNIST
In this notebook we'll be implementing a basic feed forward neural network for classifying hand-written digits from the MNIST dataset.

No GPU required!

The basic 7 steps for building models in general are listed so:
 1. Load Dataset
 2. Make Dataset Iterable
 3. Create Model Class
 4. Instantiate Model Class
 5. Instantiate Loss Class
 6. Instantiate Optimizer Class
 7. Train Model

But before we build our model lets handle set up.

`torchvision.transforms` is for image transformations. This can be used for data augmentation.
The method we'll be using for now is `ToTensor()`. [Documentation](https://pytorch.org/docs/stable/torchvision/transforms.html#torchvision.transforms.ToTensor)

In [7]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dsets
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

from torch.utils.data import Dataset, DataLoader

In [8]:
'''
STEP 1: LOAD DATASET
'''
all_df = pd.read_table('list_eval_partition.txt', delim_whitespace=True)
validation_df = all_df[all_df['evaluation_status'].str.contains('val')]
validation_df = validation_df.drop(['evaluation_status'], axis=1)[0:100]

train_df = all_df[all_df['evaluation_status'].str.contains('train')]
train_df = train_df.drop(['evaluation_status'], axis=1)[0:100]

test_df = all_df[all_df['evaluation_status'].str.contains('test')]
test_df = test_df.drop(['evaluation_status'], axis=1)[0:100]

labels_df = pd.read_table('list_category_img.txt', delim_whitespace=True)
labels_df

def func(image_name):
    category_label = labels_df[labels_df['image_name'].str.match(image_name)].iloc[0]['category_label']
    return category_label

allimagesvalidation = []
labelvalidation = []
for index in range(100):
    img_filepath = validation_df.iloc[index]['image_name']
    im = Image.open(img_filepath)
    labelvalidation.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean()) / imarr.std()
    allimagesvalidation.append(imarr)

allimagestrain = []
labeltrain = []
for index in range(100):
    img_filepath = train_df.iloc[index]['image_name']
    im = Image.open(img_filepath)
    labeltrain.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean()) / imarr.std()
    allimagestrain.append(imarr)

allimagestest = []
labeltest = []
for index in range(100):
    img_filepath = test_df.iloc[index]['image_name']
    im = Image.open(img_filepath)
    labeltest.append(func(img_filepath))
    imarr = np.uint8(np.asarray(im.convert('RGB').resize((224,224))))
    imarr = (imarr - imarr.mean()) / imarr.std()
    allimagestest.append(imarr)

C:\Users\xjia\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


FileNotFoundError: [Errno 2] No such file or directory: 'img/Sheer_Pleated-Front_Blouse/img_00000003.jpg'

In [ ]:
'''
STEP 1.5: DATASET CLASS
'''
class ClothingDataset(Dataset):
    def __init__(self, data, labels):
        self.labels = labels
        self.data = data
        
    def __getitem__(self, index):
        im = self.data[index]
        
        trans = transforms.ToTensor()
        img = trans(img)
        
        label = self.labels[index]
        label = torch.LongTensor(label)
        
        return img, label

    def __len__(self):
        return len(self.labels)

In [9]:
'''
STEP 2: MAKING DATASET ITERABLE
'''
batch_size = 10

train_dataset = ClothingDataset(allimagestrain, labeltrain)
validation_dataset = ClothingDataset(allimagesvalidation, labelvalidation) 
test_dataset = ClothingDataset(allimagestest, labeltest)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size,                                             
                                           shuffle=False)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False)
validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                          batch_size=batch_size,
                                          shuffle=False)

NameError: name 'allimagestrain' is not defined

In [ ]:
'''
STEP 3: CREATE MODEL CLASS
'''
class CNNModel(nn.Module):
    def __init__(self, output_dim):
        super(ConvolutionalNeuralNetModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, kernel_size=4, stride=2, padding=2, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2, padding=1)
        self.fc = nn.Linear(51984, output_dim)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.maxpool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [ ]:
'''
STEP 4: INSTANTIATE MODEL CLASS
'''
input_dim = 28*28
hidden_dim = 100
output_dim = 50 # number of labels to train on

model = CNNModel(output_dim)

In [ ]:
'''
STEP 5: INSTANTIATE LOSS CLASS
'''
criterion = nn.CrossEntropyLoss()

In [11]:
'''
STEP 6: INSTANTIATE OPTIMIZER CLASS
'''
learning_rate = 0.1

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

NameError: name 'model' is not defined

In [12]:
iter = 0
losses = []
accuracies = []
for epoch in range(1):
    for i, (images, labels) in enumerate(train_loader):
   
        optimizer.zero_grad()
        outputs = model(images)
        labels = torch.max(labels, 1)[1] 

        loss = criterion(outputs, labels)
        loss.backward()

        optimizer.step()

        iter += 1

        if iter % 1 == 0: 
            correct = 0
            total = 0
            for images, labels in test_loader:
                outputs = model(images)
                
                useless, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)

                labels = torch.max(labels, 1)[1] 
                correct += (predicted == labels).sum()

            accuracy = 100 * correct / total
            
            accuracies.append(accuracy)
            losses.append(loss.item())
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.item(), accuracy))

losses_in_epochs = losses[0::2]
plt.xkcd();
plt.xlabel('Epoch #');
plt.ylabel('Loss');
plt.plot(losses_in_epochs);
plt.show();

NameError: name 'CNNModel' is not defined